# Proyecto SCP
 * [X] Jared Evander Fuentes Sánchez
 * [X] Vicente Garza Reyna
 * [X] Sebastián Terrazas Santillana

In [4]:
import cplex

In [ ]:
# Lectura de Datos


In [ ]:
# Inicio del problema
problem = new cplex.Cplex()